In [ ]:
# default_exp filter

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
# stellt sicher, dass beim verändern der core library diese wieder neu geladen wird
%load_ext autoreload
%autoreload 2

# filter_and_partition

## Basic Settings

In [ ]:
# imports
from bfh_mt_hs2020_sec_data.core import get_spark_session # initialze spark
from pathlib import Path
from typing import List, Tuple, Union, Set
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import col

import pandas as pd

import shutil          # provides high level file operations
import time            # used to measure execution time
import os
import sys

In [ ]:
all_parquet_folder = "D:/data/parquet/"
all_filtered_folder = "D:/data/parq_filtered"

In [ ]:
# init Spark
spark = get_spark_session() # Session anlegen
spark # display the moste important information of the session

## 01_Load_Whole_Dataset

In [ ]:
def load_all_data(): 
    start = time.time()
    df_all = spark.read.parquet(all_parquet_folder).cache()
    duration = time.time() - start
    print("duration: ", duration)
    return df_all

## 02_Filter_Dataset

In [ ]:
def filter_whole_ds(df_all):    
    start = time.time()

    filter_string = "stmt is not null and version NOT LIKE '00%' and form in ('10-K', '10-Q')"
    df_all_filtered = df_all.where(filter_string)

    df_all_cik_exchange = df_all_filtered[['cik','exchange']].distinct() \
        .where("exchange in ('NASDAQ','NYSE','NYSE ARCA','NYSE MKT') ").selectExpr("cik", "1 as cik_select").distinct().cache()
    df_all_filter_complete = df_all_filtered.join(df_all_cik_exchange, 'cik', "left").where("cik_select == 1")

    df_all_partioned = df_all_filter_complete.repartition(16,col("cik"), col("stmt"))

    df_all_partioned.write.parquet(all_filtered_folder)
    duration = time.time() - start
    print("duration: ", duration)

## 99_Execute

In [ ]:
shutil.rmtree(all_filtered_folder,  ignore_errors=True)

In [ ]:
df_all = load_all_data()
filter_whole_ds(df_all)

In [ ]:
spark.stop()